## Direct Method for Solving Linear Equations

In [111]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

## Linear Equations
$$\begin{aligned}2x_1 &+ x_2&+ 4x_3 &= 1\\
x_1 &+ 2x_2&+ 3x_3 &= 1.5 \\
4x_1 &- x_2&+ 2x_3 &= 2\end{aligned}$$

In [112]:
data = np.array([[2, 1, 4], [1, 2, 3], [4, -1, 2]], dtype=np.float32)
hasil = np.array([[1],[1.5],[2]], dtype=np.float32)
#print("data:\n", data)
#print("hasil:\n", hasil)

In [113]:
data2 = np.array([[1, 2], [1.05, 2]])
hasil2 = np.array([[10], [10.4]])

In [114]:
data3 = np.array([[1, 2, 5, -1, 7],
                  [0, 0, 33, 2, 15],
                  [0, -4, 5, 6, 1],
                  [0, 6, 25, 99, 2],
                  [0, -8, 5, 0, 10]], dtype=np.float32)
hasil3 = np.array([[1],[2],[3],[4],[5]], dtype=np.float32)

In [115]:
data4 = np.array([[1, 1], [2, 100000]], dtype=np.float32)
hasil4 = np.array([[2], [100000]], dtype=np.float32)

## Forward Elimination

In [116]:
def forward_elimination(data, hasil):
    n, _ = hasil.shape
    #n = 3
    a = np.copy(data)
    b = np.copy(hasil)

    for k in range(n-1): ## 0|1
        for i in range(k+1,n): ## 1,2|2
            factor = a[i,k]/a[k,k]
            # fixing from j=k+1 to j=k because first must be 0     
            for j in range(k,n): ##0,1,2|1,2
                a[i,j]=a[i,j] - factor*a[k,j]
            b[i] = b[i] - factor*b[k]
    
    return a, b

Partial Pivoting to Switch row matrix
$$\text{partial pivoting when pivot is under } 10^{-1}$$

In [117]:
def partial_pivoting(data, nrow1, nrow2):
    temp = np.copy(data[nrow1,:])
    data[nrow1,:] = np.copy(data[nrow2,:])
    data[nrow2,:] = temp
    return data

Scaling to maximum value 1

In [118]:
def scale(data, baris):
    out = np.copy(data)
    val = np.max(data[baris,:])
    out[baris,:] = out[baris,:]/val
    return out

## Modified Forward Elimination 2 :
- with partial pivoting and scaling feature

In [119]:
def forward_elimination2(data, hasil, pivot_when_zero=False, scale_en=False):
    n, _ = hasil.shape
    #n = 3
    a = np.copy(data)
    b = np.copy(hasil)

    for k in range(n-1): ## 0|1
        for i in range(k+1,n): ## 1,2|2
            if (scale_en):
                a = scale(a,i)
                b = scale(b,i)
                print("scale:(k:{},i:{})\n{}\n{}".format(k,i,a,b))
            if (abs(a[k,k])<=10**-1 and pivot_when_zero==True) or pivot_when_zero==False:  ## check if zero
                index = np.argmax(np.abs(a[k:n,k]))
                index = index+k
                if (index!=k):
                    a = partial_pivoting(a,k,index)
                    b = partial_pivoting(b,k,index)
                    print("pivoting:(index:{}<->{})\n{}".format(k,index,a))
            if abs(a[i,k])>=10**-10: ## check if the number is not zero
                factor = a[i,k]/a[k,k]
                # fixing from j=k+1 to j=k because first must be 0     
                for j in range(k,n): ##0,1,2|1,2
                    a[i,j]=a[i,j] - factor*a[k,j]
                b[i] = b[i] - factor*b[k]
                print("elimination(k:{},i:{}) factor {}\n{}\n{}".format(k, i, factor, a, b))
    
    return a, b

## Back Substitution

In [120]:
def backward_substitution(a, b, round_x=3):
    n, _ = b.shape ## 3

    x = np.zeros(shape=n)

    x[n-1] = np.round(b[n-1]/a[n-1,n-1],round_x)
    for i in range(n-2,-1,-1): ## downto 1|0
        sum  = 0
        for j in range(i+1,n): ## 2|1,2
            sum = sum + a[i,j]*x[j]
        x[i] = np.round((b[i]-sum)/a[i,i],round_x)
    return x

In [121]:
#a, b = forward_elimination(data, hasil)
a, b = forward_elimination2(data4, hasil4, pivot_when_zero=False, scale_en=True)
print("matrix a:\n",a)
print("matrix out:\n",b)
x = backward_substitution(a, b, round_x=3)
print("x :\n",x)

scale:(k:0,i:1)
[[1.e+00 1.e+00]
 [2.e-05 1.e+00]]
[[2.]
 [1.]]
elimination(k:0,i:1) factor 1.9999999494757503e-05
[[1.      1.     ]
 [0.      0.99998]]
[[2.     ]
 [0.99996]]
matrix a:
 [[1.      1.     ]
 [0.      0.99998]]
matrix out:
 [[2.     ]
 [0.99996]]
x :
 [1. 1.]
